In [9]:
# Get subset of ids for JRB basins.
import geopandas as gpd

gdf = gpd.read_file(r"C:\Users\LeoLo\Desktop\jrb_2.gpkg", layer="flowpaths")
# Many more layers 'divides', 'lakes', 'nexus', 'pois', 'hydrolocations', 'flowpath-attributes', 
# 'flowpath-attributes-ml', 'network', 'divide-attributes'

# print(gdf.head())
print(f"Basins in Juniata RB: {gdf.divide_id} (unique: {gdf.divide_id.nunique()})")
jrb_divide_ids = list(gdf.divide_id)

Basins in Juniata RB: 0      cat-88306
1      cat-87647
2      cat-88001
3      cat-88268
4      cat-88269
         ...    
789    cat-88404
790    cat-88405
791    cat-88318
792    cat-87405
793    cat-87639
Name: divide_id, Length: 794, dtype: object (unique: 794)


In [36]:
# Load netcdf forcing and attribute files + trim to JRB.
import xarray as xr
import numpy as np


attrs_path = r"C:\Users\LeoLo\Desktop\attributes.nc"
forc_path = r"X:\forcings.nc" #"C:\Users\LeoLo\Desktop\forcings.nc"

# Open the NetCDF and convert to DataFrame
d_a = xr.open_dataset(attrs_path)
# attrs = d_a.to_dataframe()

d_f = xr.open_dataset(forc_path)
# forc = d_f.to_dataframe()

# Display the dataset
print(d_a)


# Get the divide_id coordinate
divide_ids = d_a['divide_id'].values

# Find duplicate divide_id values
unique, counts = np.unique(divide_ids, return_counts=True)
duplicates = unique[counts > 1]
print(f"\n --------\nAttribute data has {len(duplicates)} duplicate divide_id values.")


# Find duplicate divide_id values
divide_ids = d_f['divide_id'].values
unique, counts = np.unique(divide_ids, return_counts=True)
duplicates = unique[counts > 1]
print(f"\n --------\nForcing data has {len(duplicates)} duplicate divide_id values.")

<xarray.Dataset> Size: 225MB
Dimensions:            (divide_id: 839543)
Coordinates:
  * divide_id          (divide_id) <U11 37MB 'cat-1068193' ... 'cat-3014411'
Data variables: (12/28)
    FW                 (divide_id) float64 7MB 0.06157 0.01494 ... 7.854e-07 0.0
    HWSD_clay          (divide_id) float64 7MB 14.64 29.69 25.17 ... 28.94 28.95
    HWSD_sand          (divide_id) float64 7MB 65.4 22.53 24.07 ... 26.62 26.61
    T_clay             (divide_id) float64 7MB 6.226 25.25 13.73 ... 23.0 23.0
    uparea             (divide_id) float64 7MB 110.2 10.08 211.7 ... 3.888 4.4
    T_gravel           (divide_id) float64 7MB 17.23 4.0 8.132 ... 8.0 8.0 8.0
    ...                 ...
    ETPOT_Hargr        (divide_id) float64 7MB 819.9 853.7 ... 1.061e+03
    meanTa             (divide_id) float64 7MB 3.293 5.532 5.988 ... 6.095 6.084
    SoilGrids1km_clay  (divide_id) float64 7MB 14.81 23.31 21.97 ... 24.38 24.38
    snow_fraction      (divide_id) float64 7MB 0.2327 0.2647 ... 0.05644

,FW,HWSD_clay,HWSD_sand,T_clay,uparea,T_gravel,meanelevation,meanP,HWSD_gravel,seasonality_P,...,HWSD_silt,meanslope,permeability,seasonality_PET,ETPOT_Hargr,meanTa,SoilGrids1km_clay,snow_fraction,aridity,NDVI
divide_id,,,,,,,,,,,,,,,,,,,,,
cat-100890,0.007916,26.025530,41.448939,4.0,44.119245,4.0,869.999678,1362.221376,4.5,0.177600,...,32.525531,10.698865,-14.050000,0.650494,828.667336,3.078015,8.687385,0.396185,0.609418,0.635147
cat-100890,0.016557,25.646412,42.207175,4.0,44.119245,4.0,758.952248,1354.689346,4.5,0.147528,...,32.146412,10.058972,-14.049843,0.652170,816.112551,3.319549,9.248475,0.392257,0.608191,0.566833


In [110]:
# Only select the divide_ids that are in the JRB, and select the first occurance of any duplicate divide_ids.
import pandas as pd


## For forcing
divide_ids = d_f['divide_id'].values

# Find the first occurrence of each divide_id
unique_indices = np.unique(divide_ids, return_index=True)[1]
first_occurrence_mask = np.zeros_like(divide_ids, dtype=bool)
first_occurrence_mask[unique_indices] = True

# Apply the mask to the dataset
unique_d_f = d_f.isel(divide_id=first_occurrence_mask)

# Subset the dataset to include only the desired divide_ids
subset_d_f = unique_d_f.sel(divide_id=jrb_divide_ids)


## For attributes
divide_ids = d_a['divide_id'].values
unique_indices = np.unique(divide_ids, return_index=True)[1]
first_occurrence_mask = np.zeros_like(divide_ids, dtype=bool)
first_occurrence_mask[unique_indices] = True

unique_d_a = d_a.isel(divide_id=first_occurrence_mask)
subset_d_a = unique_d_a.sel(divide_id=jrb_divide_ids)


## Convert to dataframe
forc = subset_d_f.to_dataframe()
attrs = subset_d_a.to_dataframe()


## Trim time to 2000-2005 (divide_id is subindexed by time)
# Ensure the second level (time) is a DatetimeIndex
forc.index = forc.index.set_levels(pd.to_datetime(forc.index.levels[1]), level=1)
start_date = '2000-01-01'
end_date = '2005-12-31'
forc = forc.loc[(slice(None), slice(start_date, end_date)), :]

# Unstack divide_id so that time is the main index
forc_unstacked = forc.unstack(level=0)  # Now columns are MultiIndex (divide_id, variable)
forc_array = forc_unstacked.to_numpy().reshape(len(forc_unstacked), len(forc_unstacked.columns.levels[0]), -1)

forc_array = np.swapaxes(forc_array, 2, 1)

f_xr = subset_d_f.to_array()
f_xr = np.swapaxes((np.swapaxes(np.swapaxes(f_xr, 1, 0), 2, 1)), 0, 1)

f_xr = f_xr[:2192,]


## Save to file
forc_path = r"C:\Users\LeoLo\Desktop\forcings_jrb"
attrs_path = r"C:\Users\LeoLo\Desktop\attributes_jrb"

np.save(forc_path, forc_array)  # (2192, 794, 3)
np.save(attrs_path, attrs.to_numpy())  # (794, 28)

# save the netcdf files
subset_d_a.to_netcdf(r"C:\Users\LeoLo\Desktop\attributes_jrb.nc")
subset_d_f.to_netcdf(r"C:\Users\LeoLo\Desktop\forcings_jrb.nc")